# Jupyter AI Integration for NeuroAI

This notebook demonstrates how to use Jupyter AI to enhance the learning experience when studying neuroscience and AI concepts. Jupyter AI integrates generative AI capabilities directly into Jupyter notebooks, allowing you to:

- Generate code examples based on neuroscience concepts
- Get explanations of complex neural network architectures
- Debug and improve code implementations
- Generate visualizations for neural data
- Summarize research papers and concepts

## Setup Instructions

To use Jupyter AI, you'll need to install the required packages:

In [ ]:
# Install Jupyter AI and dependencies
!pip install -q jupyter-ai jupyter-ai-magics openai

After installation, you need to enable the extension. In JupyterLab, this happens automatically. In a classic notebook, you may need to run:

In [ ]:
# Enable Jupyter AI magics
%load_ext jupyter_ai_magics

## Configuring Jupyter AI

Jupyter AI can be configured to use different AI models:

1. OpenAI models (requires API key)
2. Anthropic Claude models (requires API key)
3. Open source models via Hugging Face or local installation

To configure an API key, you can use environment variables or the Jupyter configuration system. For this demo, we'll use a placeholder approach:

In [ ]:
# Example configuration (for demonstration only)
# In a real setup, you would set these via environment variables
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key"

print("Note: For actual use, you'll need to configure your API keys")
print("See: https://jupyter-ai.readthedocs.io/en/latest/users/index.html")

## Using Jupyter AI with NeuroAI

Jupyter AI can be used in several ways:

1. Using cell magics with `%%ai`
2. Using line magics with `%ai`
3. Using the chat interface in JupyterLab

Let's see some examples of how to use these features for NeuroAI tasks.

### Example 1: Generating Code for Neuroscience Models

We can use Jupyter AI to generate code for implementing neuroscience-inspired models:

In [ ]:
%%ai gpt-4
Write a PyTorch implementation of a predictive coding network for visual inputs, based on the PredNet architecture described in Chapter 20. Include comments explaining the biological inspiration for each component.

Note: In a live Jupyter AI environment, the code above would produce a complete PyTorch implementation of PredNet. Since we don't have API keys configured here, we'll show an example of what the output might look like:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PredNetLayer(nn.Module):
    """
    A single layer of the PredNet architecture
    
    Biological inspiration:
    - Representation units: Similar to neural populations in visual cortex that encode features
    - Prediction units: Analogous to top-down feedback connections in visual hierarchy
    - Error units: Mimics error-signaling neurons that respond to unexpected stimuli
    """
    def __init__(self, input_channels, representation_channels, prediction_channels):
        super(PredNetLayer, self).__init__()
        self.input_channels = input_channels
        self.representation_channels = representation_channels
        
        # Error computation units (biological: error-signaling neurons)
        # Separate units for positive and negative errors mimics ON/OFF pathways
        self.error_conv_pos = nn.Conv2d(input_channels, representation_channels, 
                                        kernel_size=3, padding=1)
        self.error_conv_neg = nn.Conv2d(input_channels, representation_channels, 
                                        kernel_size=3, padding=1)
        
        # Prediction units (biological: feedback connections)
        self.prediction_conv = nn.Conv2d(representation_channels, input_channels, 
                                         kernel_size=3, padding=1)
        
        # Representation update network (biological: integration in pyramidal neurons)
        self.representation_update = nn.Conv2d(2*representation_channels, 
                                             representation_channels,
                                             kernel_size=3, padding=1)
        
        # Pooling for feeding to next layer (biological: hierarchical processing)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, input_tensor, representation_tensor, top_down_tensor=None):
        # Generate predictions from current representation
        predictions = self.prediction_conv(representation_tensor)
        
        # Calculate prediction errors (biological: error-signaling neurons)
        error = input_tensor - predictions
        error_pos = F.relu(error)  # Positive errors (ON pathway)
        error_neg = F.relu(-error)  # Negative errors (OFF pathway)
        
        # Process errors through convolutional layers
        error_pos_processed = self.error_conv_pos(error_pos)
        error_neg_processed = self.error_conv_neg(error_neg)
        
        # Combine error signals (biological: integration of error signals)
        combined_error = torch.cat([error_pos_processed, error_neg_processed], dim=1)
        
        # Update representation (biological: synaptic update based on errors)
        representation_update = self.representation_update(combined_error)
        updated_representation = representation_tensor + representation_update
        
        # If we have top-down input, integrate it (biological: top-down modulation)
        if top_down_tensor is not None:
            updated_representation = updated_representation + top_down_tensor
        
        # Pool for next layer (biological: increased receptive field sizes in hierarchy)
        pooled_representation = self.pool(updated_representation)
        
        return error, updated_representation, pooled_representation

class PredNet(nn.Module):
    """
    Complete PredNet architecture with multiple layers
    
    Biological inspiration:
    - Hierarchical structure: Mimics the visual cortical hierarchy (V1, V2, V4, IT)
    - Bidirectional processing: Combines bottom-up and top-down information flow
    - Prediction error emphasis: Focuses learning on surprising/unexpected information
    """
    def __init__(self, input_channels, layer_channels):
        super(PredNet, self).__init__()
        self.num_layers = len(layer_channels)
        
        # Create PredNet layers
        self.layers = nn.ModuleList()
        in_channels = input_channels
        for i, out_channels in enumerate(layer_channels):
            self.layers.append(PredNetLayer(in_channels, out_channels, out_channels))
            in_channels = out_channels
        
        # Upsampling for top-down connections (biological: feedback projections)
        self.upsample = nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.upsample.append(nn.ConvTranspose2d(
                layer_channels[i+1], layer_channels[i],
                kernel_size=4, stride=2, padding=1
            ))
    
    def forward(self, input_sequence):
        batch_size, seq_length, channels, height, width = input_sequence.shape
        
        # Initialize representations for each layer
        representations = [torch.zeros(batch_size, layer.representation_channels, 
                                     height // (2**i), width // (2**i),
                                     device=input_sequence.device)
                         for i, layer in enumerate(self.layers)]
        
        # Initialize errors for each layer
        errors = [torch.zeros(batch_size, layer.input_channels,
                            height // (2**i), width // (2**i),
                            device=input_sequence.device)
                 for i, layer in enumerate(self.layers)]
        
        all_errors = []  # Store errors for all time steps
        
        # Process sequence
        for t in range(seq_length):
            current_input = input_sequence[:, t]
            frame_errors = []
            
            # Bottom-up pass (biological: feedforward processing)
            for i, layer in enumerate(self.layers):
                # Set input for each layer
                if i == 0:
                    layer_input = current_input
                else:
                    # Input from errors of previous layer (biological: error propagation)
                    layer_input = errors[i-1]
                
                # Process through layer
                error, updated_rep, pooled_rep = layer(layer_input, representations[i])
                
                # Store results
                errors[i] = error
                representations[i] = updated_rep
                frame_errors.append(error.mean().item())
                
                # Set input for next layer
                if i < self.num_layers - 1:
                    next_layer_input = pooled_rep
            
            # Top-down pass (biological: feedback processing)
            for i in reversed(range(self.num_layers - 1)):
                # Get top-down signal
                top_down = self.upsample[i](representations[i+1])
                
                # Update representation with top-down input
                representations[i] = representations[i] + top_down
                
            all_errors.append(frame_errors)
        
        return errors, representations, all_errors

# Example usage
def create_prednet_model():
    """Create a PredNet model for video prediction"""
    # Input: RGB images
    input_channels = 3
    # Layer channels increase with depth (analogous to increasing feature complexity)
    layer_channels = [32, 64, 128, 256]
    
    model = PredNet(input_channels, layer_channels)
    return model

# Create model
model = create_prednet_model()
print(f"PredNet model created with {sum(p.numel() for p in model.parameters())} parameters")

### Example 2: Getting Explanations of Complex Concepts

Jupyter AI is useful for getting on-demand explanations of complex neuroscience or AI concepts:

In [ ]:
%%ai anthropic/claude-3-sonnet
Explain the connection between hippocampal replay during sleep and Prioritized Experience Replay in reinforcement learning. Include key similarities and differences.

Example output (what Jupyter AI would generate with proper configuration):

**Hippocampal Replay vs. Prioritized Experience Replay**

**Similarities:**

1. **Selective Memory Processing**: Both mechanisms selectively revisit past experiences rather than processing all memories equally.
   
2. **Learning Efficiency**: Both enhance learning efficiency by focusing computational resources on high-value experiences.
   
3. **Information Consolidation**: Both help stabilize and integrate new knowledge with existing memories.
   
4. **Pattern Reactivation**: Both involve reactivating patterns of activity associated with previous experiences.
   
5. **Stability-Plasticity Balance**: Both help address the stability-plasticity dilemma by allowing new learning without catastrophic forgetting.

**Differences:**

1. **Priority Mechanism**: 
   - Hippocampal replay often prioritizes experiences based on emotional salience, novelty, reward prediction errors, and goal relevance.
   - PER explicitly uses TD errors (prediction errors) as the priority metric.

2. **Temporal Structure**:
   - Hippocampal replay occurs primarily during sleep and rest periods, with specific patterns during different sleep stages.
   - PER operates continuously during the learning process, interleaved with new experiences.

3. **Speed and Compression**:
   - Hippocampal replay often occurs at compressed timescales (5-20x faster than real experiences).
   - PER doesn't explicitly model temporal compression.

4. **Sequence Structure**:
   - Hippocampal replay often maintains sequential structure, replaying experiences in forward or reverse order.
   - PER samples individual transitions without preserving sequences.

5. **Biological Implementation**:
   - Hippocampal replay involves coordinated oscillations (sharp-wave ripples) and specific neurochemical environments.
   - PER is an algorithmic approximation that captures the functional role but not the biological implementation details.

### Example 3: Debugging and Improving Code

Jupyter AI can help debug and improve code implementations:

In [ ]:
# Example code with a bug
def simulate_hodgkin_huxley(I_ext=10, T=100, dt=0.01):
    """Simulate the Hodgkin-Huxley model of action potential generation"""
    import numpy as np
    
    # Constants
    E_Na = 55.0   # mV
    E_K = -77.0   # mV
    E_L = -65.0   # mV
    g_Na = 120.0  # mS/cm^2
    g_K = 36.0    # mS/cm^2
    g_L = 0.3     # mS/cm^2
    C_m = 1.0     # uF/cm^2
    
    # Simulation parameters
    steps = int(T / dt)
    t = np.arange(0, T, dt)
    
    # State variables
    V = np.zeros(steps)
    m = np.zeros(steps)
    h = np.zeros(steps)
    n = np.zeros(steps)
    
    # Initialize
    V[0] = -65.0  # mV
    m[0] = 0.05
    h[0] = 0.6
    n[0] = 0.32
    
    # Helper functions for gating variables
    def alpha_m(V):
        return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))
    
    def beta_m(V):
        return 4.0 * np.exp(-(V + 65.0) / 18.0)
    
    def alpha_h(V):
        return 0.07 * np.exp(-(V + 65.0) / 20.0)
    
    def beta_h(V):
        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))
    
    def alpha_n(V):
        return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))
    
    def beta_n(V):
        return 0.125 * np.exp(-(V + 65.0) / 80.0)
    
    # Bug: division by zero in alpha_m when V = -40.0
    # Bug: division by zero in alpha_n when V = -55.0
    
    # Simulation loop
    for i in range(1, steps):
        # Calculate gating variables
        m[i] = m[i-1] + dt * (alpha_m(V[i-1]) * (1 - m[i-1]) - beta_m(V[i-1]) * m[i-1])
        h[i] = h[i-1] + dt * (alpha_h(V[i-1]) * (1 - h[i-1]) - beta_h(V[i-1]) * h[i-1])
        n[i] = n[i-1] + dt * (alpha_n(V[i-1]) * (1 - n[i-1]) - beta_n(V[i-1]) * n[i-1])
        
        # Calculate conductances and currents
        g_Na_t = g_Na * m[i]**3 * h[i]
        g_K_t = g_K * n[i]**4
        
        I_Na = g_Na_t * (V[i-1] - E_Na)
        I_K = g_K_t * (V[i-1] - E_K)
        I_L = g_L * (V[i-1] - E_L)
        
        # Update membrane potential
        dV = (I_ext - I_Na - I_K - I_L) / C_m
        V[i] = V[i-1] + dt * dV
        
    return t, V, m, h, n

print("Code with potential bugs defined")

In [ ]:
%%ai gpt-4
There's a bug in my Hodgkin-Huxley model implementation. The code can cause division by zero errors in the alpha_m and alpha_n functions. Can you fix these issues and explain the neurobiological meaning of these functions?

Example output (what Jupyter AI would generate with proper configuration):

In [ ]:
def simulate_hodgkin_huxley(I_ext=10, T=100, dt=0.01):
    """Simulate the Hodgkin-Huxley model of action potential generation"""
    import numpy as np
    
    # Constants
    E_Na = 55.0   # mV
    E_K = -77.0   # mV
    E_L = -65.0   # mV
    g_Na = 120.0  # mS/cm^2
    g_K = 36.0    # mS/cm^2
    g_L = 0.3     # mS/cm^2
    C_m = 1.0     # uF/cm^2
    
    # Simulation parameters
    steps = int(T / dt)
    t = np.arange(0, T, dt)
    
    # State variables
    V = np.zeros(steps)
    m = np.zeros(steps)
    h = np.zeros(steps)
    n = np.zeros(steps)
    
    # Initialize
    V[0] = -65.0  # mV
    m[0] = 0.05
    h[0] = 0.6
    n[0] = 0.32
    
    # Helper functions for gating variables with fixed division by zero issues
    def alpha_m(V):
        # Using L'Hôpital's rule for the case when V is close to -40
        if np.abs(V + 40.0) < 1e-6:
            return 1.0  # Limit value when V = -40
        return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))
    
    def beta_m(V):
        return 4.0 * np.exp(-(V + 65.0) / 18.0)
    
    def alpha_h(V):
        return 0.07 * np.exp(-(V + 65.0) / 20.0)
    
    def beta_h(V):
        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))
    
    def alpha_n(V):
        # Using L'Hôpital's rule for the case when V is close to -55
        if np.abs(V + 55.0) < 1e-6:
            return 0.1  # Limit value when V = -55
        return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))
    
    def beta_n(V):
        return 0.125 * np.exp(-(V + 65.0) / 80.0)
    
    # Simulation loop
    for i in range(1, steps):
        # Calculate gating variables
        m[i] = m[i-1] + dt * (alpha_m(V[i-1]) * (1 - m[i-1]) - beta_m(V[i-1]) * m[i-1])
        h[i] = h[i-1] + dt * (alpha_h(V[i-1]) * (1 - h[i-1]) - beta_h(V[i-1]) * h[i-1])
        n[i] = n[i-1] + dt * (alpha_n(V[i-1]) * (1 - n[i-1]) - beta_n(V[i-1]) * n[i-1])
        
        # Calculate conductances and currents
        g_Na_t = g_Na * m[i]**3 * h[i]
        g_K_t = g_K * n[i]**4
        
        I_Na = g_Na_t * (V[i-1] - E_Na)
        I_K = g_K_t * (V[i-1] - E_K)
        I_L = g_L * (V[i-1] - E_L)
        
        # Update membrane potential
        dV = (I_ext - I_Na - I_K - I_L) / C_m
        V[i] = V[i-1] + dt * dV
        
    return t, V, m, h, n

print("Fixed code defined")

**Neurobiological Meaning of Alpha and Beta Functions**

In the Hodgkin-Huxley model, the alpha (α) and beta (β) functions represent voltage-dependent rate constants that control the opening and closing of ion channels:

1. **Alpha Functions (α_m, α_h, α_n)**: 
   - Represent the rate at which the corresponding gates transition from closed to open states
   - Neurobiologically, they model the voltage-dependent probability of conformational changes in ion channel proteins that lead to channel opening
   - For example, α_m increases with depolarization, representing faster sodium channel activation as voltage increases

2. **Beta Functions (β_m, β_h, β_n)**:
   - Represent the rate at which the gates transition from open to closed states
   - Neurobiologically, they model the rate at which ion channels return to their resting conformation
   - For example, β_h increases with depolarization, representing faster sodium channel inactivation

3. **Specific Gating Variables**:
   - **m**: Controls sodium channel activation (opens quickly with depolarization)
   - **h**: Controls sodium channel inactivation (closes slowly with sustained depolarization)
   - **n**: Controls potassium channel activation (opens slowly with depolarization)

The fixes I implemented use L'Hôpital's rule to handle the special cases where the denominator becomes zero, which happens at specific voltage values (-40 mV for α_m and -55 mV for α_n). These special cases are important physiologically as they represent critical voltages where channel opening rates undergo significant changes.

### Example 4: Generating and Improving Visualizations

Jupyter AI can help create and enhance visualizations of neural data:

In [ ]:
%%ai anthropic/claude-3-opus
Generate code to visualize the attention maps in a Vision Transformer model, similar to what we discussed in Chapter 20. The visualization should show both the image and the corresponding attention patterns between patches. Include interactivity to explore different attention heads.

Example output (what Jupyter AI would generate with proper configuration):

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import ipywidgets as widgets
from ipywidgets import interact, fixed
from PIL import Image
import requests
from io import BytesIO
from transformers import ViTModel, ViTImageProcessor

# Function to load an image from URL or file path
def load_image(image_path):
    """Load image from URL or file path"""
    if image_path.startswith('http'):
        response = requests.get(image_path)
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(image_path)
    return img

# Function to extract attention maps from ViT model
def get_attention_maps(model, processor, image):
    """Extract attention maps from Vision Transformer"""
    # Preprocess image
    inputs = processor(images=image, return_tensors="pt")
    
    # Forward pass with output_attentions=True
    outputs = model(**inputs, output_attentions=True)
    
    # Get attention maps [layers, heads, tokens, tokens]
    attention_maps = outputs.attentions
    
    return attention_maps, inputs.pixel_values

# Function to visualize attention maps
def visualize_attention(attention_maps, image, layer, head, patch_size=16):
    """Visualize attention maps for a specific layer and attention head"""
    # Get the specified attention map
    attention = attention_maps[layer][0, head].detach().numpy()
    
    # The first token is the [CLS] token, so we exclude it for visualization
    attention = attention[0, 1:]
    
    # Reshape attention to match image patches
    num_patches = int(np.sqrt(attention.shape[0]))
    attention_map = attention.reshape(num_patches, num_patches)
    
    # Create a figure with subplots
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))
    
    # Plot original image
    axs[0].imshow(image)
    axs[0].set_title("Original Image")
    axs[0].axis('off')
    
    # Create grid to show patches
    h, w = image.size[1], image.size[0]
    for i in range(0, h, patch_size):
        axs[0].axhline(i, color='blue', lw=0.5, alpha=0.5)
    for j in range(0, w, patch_size):
        axs[0].axvline(j, color='blue', lw=0.5, alpha=0.5)
    
    # Highlight the focus patch (CLS token attends to this patch)
    focus_i, focus_j = 0, 0  # By default, use the top-left patch
    rect = plt.Rectangle((focus_j, focus_i), patch_size, patch_size,
                         edgecolor='red', facecolor='none', lw=2)
    axs[0].add_patch(rect)
    
    # Plot attention map
    # Create a custom colormap with better contrast
    cmap = LinearSegmentedColormap.from_list('attention_cmap', ['#0000ff', '#00ffff', '#ffff00', '#ff0000'])
    im = axs[1].imshow(attention_map, cmap=cmap)
    axs[1].set_title(f"Attention Map (Layer {layer+1}, Head {head+1})")
    fig.colorbar(im, ax=axs[1], shrink=0.7, label='Attention Strength')
    
    # Add grid to match patches
    axs[1].set_xticks(np.arange(-0.5, num_patches, 1), minor=True)
    axs[1].set_yticks(np.arange(-0.5, num_patches, 1), minor=True)
    axs[1].grid(which='minor', color='black', linestyle='-', linewidth=0.5, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Interactive visualization function
def interactive_attention_visualization(image_path):
    """Create interactive widgets to explore ViT attention maps"""
    # Load model and processor
    print("Loading Vision Transformer model...")
    model = ViTModel.from_pretrained('google/vit-base-patch16-224')
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
    
    # Load image
    image = load_image(image_path)
    
    # Get attention maps
    print("Extracting attention maps...")
    attention_maps, _ = get_attention_maps(model, processor, image)
    
    # Number of layers and heads
    num_layers = len(attention_maps)
    num_heads = attention_maps[0].shape[1]
    
    print(f"Model has {num_layers} layers with {num_heads} attention heads each")
    
    # Create interactive widgets
    layer_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=num_layers-1,
        step=1,
        description='Layer:',
        continuous_update=False
    )
    
    head_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=num_heads-1,
        step=1,
        description='Head:',
        continuous_update=False
    )
    
    # Create interactive visualization
    interact(
        lambda layer, head: visualize_attention(attention_maps, image, layer, head),
        layer=layer_slider,
        head=head_slider
    )

# Example usage - enable this to run the interactive visualization
# interactive_attention_visualization('https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo/bobcat.jpg')

print("Vision Transformer attention visualization code defined")

## Adding Jupyter AI to the Handbook

To fully integrate Jupyter AI into the NeuroAI Handbook, we should follow these steps:

1. **Installation Instructions**: Add clear instructions for users to install and configure Jupyter AI
2. **Configuration Guide**: Provide guidance on setting up API keys for different models
3. **Usage Examples**: Add examples for common NeuroAI tasks (like those shown above)
4. **Integration with Exercises**: Enhance the existing exercises with AI-assisted options

Let's create a mini-guide for users:

## Using Jupyter AI with the NeuroAI Handbook

Jupyter AI can significantly enhance your learning experience by providing:

- On-demand explanations of complex neuroscience concepts
- Code generation for implementing models discussed in the handbook
- Help with debugging and improving your implementations
- Support for visualizing neural data and model architectures

### Quick Start Guide

1. Install the required packages:
   ```bash
   pip install jupyter-ai jupyter-ai-magics openai
   ```

2. Configure your API keys (we recommend using environment variables):
   ```bash
   # For OpenAI models
   export OPENAI_API_KEY="your-api-key"
   
   # For Anthropic models
   export ANTHROPIC_API_KEY="your-api-key"
   ```

3. Load the extension in your notebook:
   ```python
   %load_ext jupyter_ai_magics
   ```

4. Use the `%%ai` magic to interact with AI models:
   ```python
   %%ai gpt-4
   Explain how the PredNet architecture mimics predictive coding in the visual cortex.
   ```

### Recommended Use Cases

1. **Code Assistance**: Get help implementing algorithms from the handbook
2. **Concept Clarification**: Ask for detailed explanations of key concepts
3. **Exploration**: Ask "what if" questions about model variations
4. **Visualization**: Generate code for visualizing model architectures or neural data
5. **Debugging**: Get help diagnosing and fixing issues in your implementations

### Available Models

- **OpenAI**: gpt-3.5-turbo, gpt-4
- **Anthropic**: anthropic/claude-3-sonnet, anthropic/claude-3-opus
- **Open Source**: Various models via Hugging Face

For more detailed information, see the [Jupyter AI documentation](https://jupyter-ai.readthedocs.io/).

## Conclusion

Jupyter AI provides powerful AI-assisted features that can significantly enhance the interactive learning experience of the NeuroAI Handbook. By integrating these capabilities, readers can:

1. Get personalized explanations of complex neuroscience and AI concepts
2. Generate and debug code for implementing models discussed in the handbook
3. Create visualizations to better understand neural data and model architectures
4. Explore variations and extensions of the models presented in the case studies

This integration supports a more dynamic, inquiry-based learning approach that can adapt to each reader's specific interests and questions.